In [67]:
!pip install scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 12.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 5.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 11.7 MB/s eta 0:00:0000:0100:01


In [509]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np

In [510]:
df = pd.read_parquet('yellow_tripdata_2023-01.parquet')

In [511]:
list(df.columns)

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'airport_fee']

In [512]:
len(df.columns)

19

In [513]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [514]:
df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']

In [515]:
df['duration'] = pd.to_timedelta(df['duration'])

In [516]:
df['duration'].std()

Timedelta('0 days 00:42:35.661074517')

In [517]:
df_filtered = df[(df['duration'] >= pd.Timedelta(minutes=1)) & (df['duration'] <= pd.Timedelta(minutes=60))]

In [518]:
df_filtered

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,0 days 00:08:26
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,0 days 00:06:19
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,0 days 00:12:45
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,0 days 00:09:37
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,0 days 00:10:50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN,0 days 00:13:59
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN,0 days 00:19:27
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN,0 days 00:24:31
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN,0 days 00:13:00


## 3. Franction left after deleting the outliers

In [519]:
fraction = len(df_filtered)/len(df)
fraction*100

98.1220282212598

In [520]:
df_filtered['PULocationID']=df_filtered['PULocationID'].astype(str)
df_filtered['DOLocationID']=df_filtered['DOLocationID'].astype(str)
df_filtered['duration']=df_filtered['duration'].dt.total_seconds()

/var/folders/ct/khrrzdws0svc5w3r1kfgp69r0000gn/T/ipykernel_27876/3235349145.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['PULocationID']=df_filtered['PULocationID'].astype(str)
/var/folders/ct/khrrzdws0svc5w3r1kfgp69r0000gn/T/ipykernel_27876/3235349145.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['DOLocationID']=df_filtered['DOLocationID'].astype(str)
/var/folders/ct/khrrzdws0svc5w3r1kfgp69r0000gn/T/ipykernel_27876/3235349145.py:3: SettingWithCopyWarning: 
A value is tr

In [521]:
# he format required by DictVectorizer
# dict_list = df[['PULocationID', 'DOLocationID']].to_dict(orient='records')
categorical = ['PULocationID','DOLocationID']
numerical = ['duration']
train_dicts = df_filtered[categorical + numerical].to_dict(orient='records')
train_dicts

[{'PULocationID': '161', 'DOLocationID': '141', 'duration': 506.0},
 {'PULocationID': '43', 'DOLocationID': '237', 'duration': 379.0},
 {'PULocationID': '48', 'DOLocationID': '238', 'duration': 765.0},
 {'PULocationID': '138', 'DOLocationID': '7', 'duration': 577.0},
 {'PULocationID': '107', 'DOLocationID': '79', 'duration': 650.0},
 {'PULocationID': '161', 'DOLocationID': '137', 'duration': 738.0},
 {'PULocationID': '239', 'DOLocationID': '143', 'duration': 627.0},
 {'PULocationID': '142', 'DOLocationID': '200', 'duration': 1364.0},
 {'PULocationID': '164', 'DOLocationID': '236', 'duration': 896.0},
 {'PULocationID': '141', 'DOLocationID': '107', 'duration': 654.0},
 {'PULocationID': '234', 'DOLocationID': '68', 'duration': 524.0},
 {'PULocationID': '79', 'DOLocationID': '264', 'duration': 2021.0},
 {'PULocationID': '164', 'DOLocationID': '143', 'duration': 1781.0},
 {'PULocationID': '138', 'DOLocationID': '33', 'duration': 1194.0},
 {'PULocationID': '33', 'DOLocationID': '61', 'durat

In [522]:

dv = DictVectorizer()

In [523]:
X_train=dv.fit_transform(train_dicts)
X_train

<3009173x516 sparse matrix of type '<class 'numpy.float64'>'
	with 9027519 stored elements in Compressed Sparse Row format>

In [524]:
type(X_train)

scipy.sparse._csr.csr_matrix

In [525]:
X_train.shape

(3009173, 516)

In [526]:
y_train = df_filtered['duration'].values
y_train

array([ 506.,  379.,  765., ..., 1471.,  780.,  864.])

## What's the dimensionality of this matrix (number of columns)?


In [527]:
X_train.shape

(3009173, 516)

In [528]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [529]:
preds=model.predict(X_train)
preds

array([ 505.9999939 ,  378.99998928,  764.99999013, ..., 1470.99996862,
        780.00000141,  863.99999264])

In [530]:
y_train

array([ 506.,  379.,  765., ..., 1471.,  780.,  864.])

In [531]:
rmse_train = np.sqrt(mean_squared_error(preds, y_train))


In [532]:
rmse_train

3.349097322064995e-05

In [323]:
test_df = pd.read_parquet('yellow_tripdata_2023-02.parquet')

In [473]:
def test(test_df):
    test_df['duration'] = test_df['tpep_dropoff_datetime'] - test_df['tpep_pickup_datetime']

    test_df['duration'] = pd.to_timedelta(test_df['duration'])

    test_df = test_df[(test_df['duration'] >= pd.Timedelta(minutes=1)) & (test_df['duration'] <= pd.Timedelta(minutes=60))]

    test_df['PULocationID']=test_df['PULocationID'].astype(str)
    test_df['DOLocationID']=test_df['DOLocationID'].astype(str)

    test_df['duration']=test_df['duration'].dt.total_seconds()

    categorical = ['PULocationID','DOLocationID']
    numerical = ['duration']
    val_dicts=test_df[categorical + numerical].to_dict(orient='records')
    y_val = test_df['duration'].values
    dv = DictVectorizer()
    val_dicts_final = dv.fit_transform(val_dicts)
    return val_dicts_final.shape


In [382]:
val_dicts=test_df[categorical].to_dict(orient='records')

In [383]:
val_dicts

[{'PULocationID': '142.0', 'DOLocationID': '163.0', 'duration': 101.0},
 {'PULocationID': '132.0', 'DOLocationID': '26.0', 'duration': 1925.0},
 {'PULocationID': '161.0', 'DOLocationID': '145.0', 'duration': 798.0},
 {'PULocationID': '148.0', 'DOLocationID': '236.0', 'duration': 878.0},
 {'PULocationID': '137.0', 'DOLocationID': '244.0', 'duration': 1677.0},
 {'PULocationID': '263.0', 'DOLocationID': '141.0', 'duration': 224.0},
 {'PULocationID': '48.0', 'DOLocationID': '243.0', 'duration': 796.0},
 {'PULocationID': '114.0', 'DOLocationID': '211.0', 'duration': 223.0},
 {'PULocationID': '114.0', 'DOLocationID': '249.0', 'duration': 315.0},
 {'PULocationID': '125.0', 'DOLocationID': '107.0', 'duration': 441.0},
 {'PULocationID': '140.0', 'DOLocationID': '42.0', 'duration': 746.0},
 {'PULocationID': '140.0', 'DOLocationID': '226.0', 'duration': 641.0},
 {'PULocationID': '249.0', 'DOLocationID': '90.0', 'duration': 271.0},
 {'PULocationID': '234.0', 'DOLocationID': '4.0', 'duration': 624.

In [384]:
y_val = test_df['duration'].values
y_val

array([ 101., 1925.,  798., ...,  840.,  420.,  588.])

In [385]:
val_dicts_final = dv.fit_transform(val_dicts)

In [386]:
val_dicts_final.shape


(2855951, 515)

In [336]:

def predict_val(df):

    df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    print(list(df.columns))
    
    df = df[(df['duration'] >= pd.Timedelta(minutes=1)) & (df['duration'] <= pd.Timedelta(minutes=60))]
    df['PULocationID']=df['PULocationID'].astype(str)
    df['DOLocationID']=df['DOLocationID'].astype(str)
    
    df['duration']=df['duration'].dt.total_seconds()
    categorical = ['PULocationID','DOLocationID']
    numerical = ['duration']
    
    y_val = df['duration'].values
    
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    
    X_val=dv.fit_transform(train_dicts)
    model = LinearRegression()
    model.fit(X_val, y_val)
    
    preds = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(preds, y_val))
    
    
    return [preds, rmse]

In [337]:
res=predict_val(test_df)


['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'Airport_fee', 'duration']


/var/folders/ct/khrrzdws0svc5w3r1kfgp69r0000gn/T/ipykernel_27876/2014359949.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PULocationID']=df['PULocationID'].astype(str)
/var/folders/ct/khrrzdws0svc5w3r1kfgp69r0000gn/T/ipykernel_27876/2014359949.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DOLocationID']=df['DOLocationID'].astype(str)
/var/folders/ct/khrrzdws0svc5w3r1kfgp69r0000gn/T/ipykernel_27876/2014359949.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [338]:
res[0]

array([ 101.00000964, 1925.00011343,  798.000002  , ...,  840.00000211,
        420.00000322,  588.00000141])

In [290]:
test_df['duration']=test_df['tpep_dropoff_datetime'] - test_df['tpep_pickup_datetime']
test_df = test_df[(test_df['duration'] >= pd.Timedelta(minutes=1)) & (test_df['duration'] <= pd.Timedelta(minutes=60))]
test_df['duration']=test_df['duration'].dt.total_seconds()

/var/folders/ct/khrrzdws0svc5w3r1kfgp69r0000gn/T/ipykernel_27876/1521066481.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['duration']=test_df['tpep_dropoff_datetime'] - test_df['tpep_pickup_datetime']


In [387]:
val = pd.read_parquet('yellow_tripdata_2023-02.parquet')
org = pd.read_parquet('yellow_tripdata_2023-01.parquet')

In [474]:
test(val)

/var/folders/ct/khrrzdws0svc5w3r1kfgp69r0000gn/T/ipykernel_27876/82840829.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['PULocationID']=test_df['PULocationID'].astype(str)
/var/folders/ct/khrrzdws0svc5w3r1kfgp69r0000gn/T/ipykernel_27876/82840829.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['DOLocationID']=test_df['DOLocationID'].astype(str)
/var/folders/ct/khrrzdws0svc5w3r1kfgp69r0000gn/T/ipykernel_27876/82840829.py:11: SettingWithCopyWarning: 
A value is trying to be set on a c

(2855951, 515)

In [475]:
test(org)

/var/folders/ct/khrrzdws0svc5w3r1kfgp69r0000gn/T/ipykernel_27876/82840829.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['PULocationID']=test_df['PULocationID'].astype(str)
/var/folders/ct/khrrzdws0svc5w3r1kfgp69r0000gn/T/ipykernel_27876/82840829.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['DOLocationID']=test_df['DOLocationID'].astype(str)
/var/folders/ct/khrrzdws0svc5w3r1kfgp69r0000gn/T/ipykernel_27876/82840829.py:11: SettingWithCopyWarning: 
A value is trying to be set on a c

(3009173, 516)

In [619]:
def test(test_df):
    test_df['duration'] = test_df['tpep_dropoff_datetime'] - test_df['tpep_pickup_datetime']
    test_df['duration'] = pd.to_timedelta(test_df['duration'])
    test_df = test_df[(test_df['duration'] >= pd.Timedelta(minutes=1)) & (test_df['duration'] <= pd.Timedelta(minutes=60))]
    test_df['PULocationID'] = test_df['PULocationID'].astype(str)
    test_df['DOLocationID'] = test_df['DOLocationID'].astype(str)
    test_df['duration'] = test_df['duration'].dt.total_seconds()
    categorical = ['PULocationID','DOLocationID']
    numerical = ['duration']
    val_dicts = test_df[categorical + numerical].to_dict(orient='records')
    val_dicts_final = dv.transform(val_dicts)
    y = test_df['duration'].values
    return val_dicts_final, y, test_df





In [629]:
result = test(val)


/var/folders/ct/khrrzdws0svc5w3r1kfgp69r0000gn/T/ipykernel_27876/2822247752.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['PULocationID'] = test_df['PULocationID'].astype(str)
/var/folders/ct/khrrzdws0svc5w3r1kfgp69r0000gn/T/ipykernel_27876/2822247752.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['DOLocationID'] = test_df['DOLocationID'].astype(str)
/var/folders/ct/khrrzdws0svc5w3r1kfgp69r0000gn/T/ipykernel_27876/2822247752.py:7: SettingWithCopyWarning: 
A value is trying to be s

In [630]:
y=result[1]
X_train = result[0]
f_df =result[2]
f_df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
0,1.0,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142.0,163.0,2.0,4.40,3.50,0.5,0.00,0.0,1.0,9.40,2.5,0.00,101.0
3,1.0,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132.0,26.0,1.0,70.90,2.25,0.5,0.00,0.0,1.0,74.65,0.0,1.25,1925.0
4,2.0,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161.0,145.0,1.0,17.00,1.00,0.5,3.30,0.0,1.0,25.30,2.5,0.00,798.0
5,1.0,2023-02-01 00:52:40,2023-02-01 01:07:18,1.0,5.10,1.0,N,148.0,236.0,1.0,21.90,3.50,0.5,5.35,0.0,1.0,32.25,2.5,0.00,878.0
6,1.0,2023-02-01 00:12:39,2023-02-01 00:40:36,1.0,8.90,1.0,N,137.0,244.0,1.0,41.50,3.50,0.5,3.50,0.0,1.0,50.00,2.5,0.00,1677.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2913950,2.0,2023-02-28 23:46:00,2023-03-01 00:05:00,NaN,4.65,NaN,None,249.0,140.0,0.0,20.22,0.00,0.5,4.84,0.0,1.0,29.06,NaN,NaN,1140.0
2913951,2.0,2023-02-28 23:26:02,2023-02-28 23:37:10,NaN,2.47,NaN,None,186.0,79.0,0.0,13.66,0.00,0.5,2.65,0.0,1.0,20.31,NaN,NaN,668.0
2913952,2.0,2023-02-28 23:24:00,2023-02-28 23:38:00,NaN,3.49,NaN,None,158.0,143.0,0.0,17.64,0.00,0.5,0.00,0.0,1.0,21.64,NaN,NaN,840.0
2913953,2.0,2023-02-28 23:03:00,2023-02-28 23:10:00,NaN,2.13,NaN,None,79.0,162.0,0.0,13.56,0.00,0.5,2.63,0.0,1.0,20.19,NaN,NaN,420.0


In [624]:
model_1 = LinearRegression()

In [625]:

model_1.fit(X_train, y)

LinearRegression()

In [631]:
pre=model_1.predict(X_train)

In [632]:
rmse_train = np.sqrt(mean_squared_error(pre, y))

In [633]:
rmse_train

2.1467707211037866e-05

In [613]:
pre=model_1.predict(X_train)

In [634]:
pre

array([ 101.00003013, 1924.99996599,  798.00000562, ...,  840.00000414,
        420.00001891,  588.000013  ])

In [636]:
rmse_train = np.sqrt(mean_squared_error(pre, y))
rmse_train

2.1467707211037866e-05